In [237]:
import pandas as pd
import time
import mysql.connector

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [204]:
df = pd.read_csv("ttrain.csv", encoding="UTF-8")
df = df.iloc[:, 1:]
df.columns = ["dept", "arr", "category", "cases"]
df[["dept", "arr"]] = df[["dept", "arr"]].apply(lambda x: x.astype(str))
df[["dept", "arr"]] = df[["dept", "arr"]].apply(lambda x: x.str[:5])

code = pd.read_excel("code.xlsx", sheet_name=0)
code.columns = ["code"]
code['code'] = code['code'].apply(lambda x: x[11:])
code = code['code'].str.split(' | ', expand=True)
code = code.drop(code.columns[[1, 3]], axis=1)
code = code.rename(columns={0 : 'code',2 : 'name'})
code['name'] = code['name'].astype(str)

df_merged = pd.merge(df, code, left_on="dept", right_on="code", how="left")
df_merged.rename(columns={"name": "dept_name"}, inplace=True)
df_merged = pd.merge(df_merged, code, left_on="arr", right_on="code", how="left")
df_merged.rename(columns={"name": "arr_name"}, inplace=True)
df = df_merged[["dept", "dept_name", "arr", "arr_name", "category", "cases"]]
df = df.groupby(["dept_name", "arr_name", "category"]).agg(total_cases=("cases", "sum")).reset_index()

In [152]:
name = input("원하는 지역? ")

원하는 지역?  파주시


In [189]:
# 첫번째 테이블
dept_region_subset = df[df["dept_name"].str.contains(name)]
dept_region_subset.head(3)

,dept_name,arr_name,category,total_cases
4108,파주시,서귀포시,가공식품,147
4109,파주시,서귀포시,기타디지털/가전,7
4110,파주시,서귀포시,기타식품,9


In [190]:
category_sum = df[df["dept_name"].str.contains(name)].groupby("category")\
.agg(total_cases=("total_cases", "sum"))\
.sort_values("total_cases", ascending=False)\
.head(3)\
.reset_index()
category_sum["dept_region"] = name
category_sum

,category,total_cases,dept_region
0,문화컨텐츠,4273,파주시
1,음반,672,파주시
2,가공식품,464,파주시


In [191]:
top_categories = category_sum["category"].tolist()
top_categories

['문화컨텐츠', '음반', '가공식품']

In [192]:
category_sum = df[df["arr_name"].str.contains(name)].groupby("category")\
.agg(total_cases=("total_cases", "sum"))\
.sort_values("total_cases", ascending=False)\
.head(3)\
.reset_index()
category_sum["arr_region"] = name
category_sum

,category,total_cases,arr_region
0,농산물,684,파주시
1,기타식품,56,파주시
2,음료,52,파주시


In [193]:
top_categories = category_sum["category"].tolist()
top_categories

['농산물', '기타식품', '음료']

In [199]:
def getitem(top_categories):
    chrome_options = Options()
    driver = webdriver.Chrome(service=Service('/Users/gwagdoseong/Documents/Rprogramming/chromedriver.exe'), options=chrome_options)

    max_page = 5
    result = pd.DataFrame(columns=["검색어", "상품명"])

    for word in top_categories:
        for i in range(1, max_page+1):
            URL = f'https://search.shopping.naver.com/search/all?query={word}&pagingIndex={i}&pagingSize=40&productSet=total&sort=rel'
            driver.get(URL)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            goods_list = soup.select('div.basicList_title__VfX3c')

            item_names = []
            for v in goods_list:
                item_name = v.select_one('div.basicList_title__VfX3c > a').get('title')
                item_names.append(item_name)

            df = pd.DataFrame({"검색어": [word]*len(item_names), "상품명": item_names})
            result = pd.concat([result, df], ignore_index=True)
    return result
    driver.quit()

In [200]:
results = getitem(top_categories)

In [ ]:
results

,검색어,상품명
0,농산물,유럽 샐러드 채소 쌈야채1Kg 무농약 유러피안
1,농산물,[첫구매한정]영양쌀 (혼합/보통) 20kg
2,농산물,"낙산배월리농원 7.5 KG(10과,11과,12과)"
3,농산물,수향미 골든퀸 10kg
4,농산물,사과 경북 부사 못난이 꿀사과 5kg 10kg
...,...,...
70,음료,Dole 돌쥬스 팩음료 파인애플 주스
71,음료,슈가로로 투명콜라 제로코크 라임콜라 350ml
72,음료,ES키키 감귤 레진 1.8L
73,음료,롯데칠성음료 펩시콜라 245ml


In [223]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="264433"
)

In [224]:
cursor = mydb.cursor()
cursor.execute("CREATE DATABASE LOGISTICS")

In [225]:
cursor.execute("SET NAMES utf8;")
cursor.execute("SET CHARACTER SET utf8;")
cursor.execute("SET character_set_connection=utf8;")

In [226]:
cursor.execute("USE LOGISTICS")

In [227]:
cursor.execute("CREATE TABLE \
logistics (dept_name VARCHAR(15)CHARACTER SET utf8,\
arr_name VARCHAR(15)CHARACTER SET utf8,\
category VARCHAR(15)CHARACTER SET utf8,\
total_cases INT) ")

In [228]:
cursor.execute("SET GLOBAL local_infile = true;")

In [231]:
for _, row in df.iterrows():
    cursor.execute(f"INSERT INTO logistics (dept_name, arr_name, category, total_cases) \
        VALUES ('{row['dept_name']}', '{row['arr_name']}', '{row['category']}', {row['total_cases']})")


In [234]:
cursor.execute("SELECT * FROM logistics")
result = cursor.fetchall()

In [236]:
cursor.close()

True